In [1]:
from packages.serial_link_robot import *
import sympy as sp
from numpy import pi

In [4]:
theta1, theta2, theta3, d1, d2, x, y = sp.symbols('theta1, theta2, theta3, d1, d2, x, y')
robot = SerialLinkRobot()
robot.add_revolute_joint(theta1, 2, 3, 0)
robot.add_revolute_joint(theta2, 0, 3, -sp.pi/2)
robot.interact()

In [4]:
theta1, d2, d3, theta4, theta5, theta6, d1, d6 = sp.symbols('theta1, d2, d3, theta4, theta5, theta6, d1, d6')
robot = SerialLinkRobot()
robot.add_revolute_joint(theta1, d1, 0, 0)
robot.add_prismatic_joint(0, d2, 0, -sp.pi/2)
robot.add_prismatic_joint(0, d3, 0, 0)
robot.add_revolute_joint(theta4, 0, 0, sp.pi/2,-3,3)
robot.add_revolute_joint(theta5, 0, 0, -sp.pi/2,-3,3)
robot.add_revolute_joint(theta6, 0, 2, 0,-3,3)
robot.add_subs([(d1, 1)])
robot.interact('ds',dynamically=False)

In [ ]:
robot = SerialLinkRobot()
robot.import_robot()
robot.interact('asd')